In [4]:
!pip install torch
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.8 MB/s eta 0:00:0000:0100:01


In [13]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [14]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker
...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,2712,66.594637,73,Male,Never smoked
1545,ID00426637202313170790466,19,2978,73.126412,73,Male,Never smoked
1546,ID00426637202313170790466,31,2908,71.407524,73,Male,Never smoked
1547,ID00426637202313170790466,43,2975,73.052745,73,Male,Never smoked


In [38]:
y = train["FVC"]
train_dl = train.copy()
train_dl=train_dl.drop(columns = ["Patient"])
train_dl['Sex'] = train_dl['Sex'].map(lambda x: 1 if(x == "Male") else 0 )
train_dl['SmokingStatus'] = train_dl['SmokingStatus'].map(lambda x: 1 if(x == "Ex-smoker") else 0 )
train_dl = train_dl.astype('double')


In [39]:
batch_size = 100
n_iters = 3000
num_epochs = 150


In [52]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x).double()

        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out).double()

        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out.double()

In [53]:
input_dim = 5
hidden_dim = 100
output_dim = 1

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()


In [54]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [55]:
num_epochs = 2

In [57]:
iter = 0
for epoch in range(num_epochs):
    model.train()
    for index, row in train_dl.iterrows():
        preds = model(torch.tensor([row['Weeks'], row['Percent'],row['Age'],row['Sex'],row['SmokingStatus']]))
        loss = loss_func(preds, torch.tensor([row['FVC']]))

        loss.backward()
        opt.step()
        opt.zero_grad()

    model.eval()

RuntimeError: expected scalar type Double but found Float